In [57]:
import pandas as pd
import datetime as dt
import numpy as np
import csv

from numpy import sin, cos, sqrt

from astropy.time import Time

import pdir

import os

## Import

In [58]:
# Script input

FILENAME = "horizons_results_earth_2019-2020.txt"
# FILENAME = "horizons_results_earth_2019-2039.txt"
# FILENAME = "horizons_results_earth_2019-2262.txt"

# FILENAME = "horizons_results_mars_2019-2020.txt"
# FILENAME = "horizons_results_mars_2019-2039.txt"
# FILENAME = "horizons_results_mars_2019-2262.txt"

limited_timespan = True  # Set false if more years than 2019-2261 are wanted, but dates will not be converted

Assumed format of filename:

`horizons_results_[BODY]_[START_YEAR]-[END-YEAR].txt`

where the dates of START_YEAR (closed interval, including last day of previous year) and END_YEAR (open interval, only including first day of year). E.g. `horizons_results_earth_2019-2020.txt` simply includes all of 2019 with one extra day in both ends, i.e. goes from `2018-12-31 00:00:00` until `2020-01-01 00:00:00` with an interval of 1 day.

Due to Timestamp conversion limitations, we don't allow dates above 2262-01-01:
```python
print(pd.Timestamp.min)  # 1677-09-21 00:12:43.145225
print(pd.Timestamp.max)  # 2262-04-11 23:47:16.854775807
```

If more years are needed, re-write script to now use `pd.to_datetime`, i.e. convert date to pandas type Timestamp.

NOTE:
Horizon limits the number of rows such that startin from 2019, 2265 is the maximum year anyway.
Also ephemerides are [available](https://ssd.jpl.nasa.gov/eph_spans.cgi?id=A) in the following time spans:

- **Earth:** B.C. 9998-Mar-20 to A.D. 9999-Dec-31
- **Mars:** 1600-Jan-01 to 2500-Jan-04 

In [59]:
# Change CDW (current working directiory)

FILE_PATH = "../orbsim/r4b_3d/ephemerides/"

cwd = os.getcwd()
in_correct_cwd = 'code' + FILE_PATH[2:-1] == cwd[-30:] # check if last part of cwd is '/code/orbsim/r4b_3d'

if not in_correct_cwd:    
    os.chdir(FILE_PATH)
    cwd = os.getcwd()

print(cwd)

/Users/gandalf/Dropbox/repositories/letomes/code/orbsim/r4b_3d/ephemerides


In [60]:
# Derived variables

_, _, BODY, DATE_RANGE = FILENAME.split('.')[0].split('_')
START_YEAR, END_YEAR = DATE_RANGE.split('-')
OUTPUT = "{}_{}.csv".format(BODY, DATE_RANGE)  # output filename

with open(FILENAME, 'r') as file:
    for i, line in enumerate(file):
        if line == "$$SOE\n":
            SOE = i + 1
        if line == "$$EOE\n":
            EOE = i + 1

FOOTER = 117
EOD = EOE + FOOTER

print("SOE: {}".format(SOE))
print("EOE: {}".format(EOE))
print("FOOTER: {}".format(FOOTER))
print("EOD: {}".format(EOD))

print("BODY: {}".format(BODY))
print("DATE_RANGE: {}".format(DATE_RANGE))
print("START_YEAR: {}".format(START_YEAR))
print("END_YEAR: {}".format(END_YEAR))
print("OUTPUT: {}".format(OUTPUT))
print("SOE: {}".format(SOE))

SOE: 62
EOE: 88819
FOOTER: 117
EOD: 88936
BODY: mars
DATE_RANGE: 2019-2262
START_YEAR: 2019
END_YEAR: 2262
OUTPUT: mars_2019-2262.csv
SOE: 62


In [61]:
# Check if date interval is valid

if int(START_YEAR) < 1678:
    raise ValueError("Ephemerides end date must be in a year no earier than 1678 (01-01), otherwise pd.to_datetime fails down below")

if int(END_YEAR) > 2262:
    raise ValueError("Ephemerides end date must be in a year no later than 2262 (01-01), otherwise pd.to_datetime fails down below")

In [62]:
# Read in raw txt file

df = pd.read_csv(FILENAME,
                 skiprows=[x for x in range(0,SOE) if x!=SOE-3], # skip all rows until SOE except for headers
                 skipfooter=FOOTER,
                 engine='python',
                 error_bad_lines=False,
                 quoting=csv.QUOTE_NONE)

df.rename(columns=lambda x: x.strip(), inplace=True)  # strip whitespace from headers
df.drop('Unnamed: 26', axis=1, inplace=True)  # dropped erroneous 'n.p.' column

df

,Date__(UT)__HR:MN:SC.fff,Date_________JDUT,,.1,R.A._(ICRF/J2000.0),DEC_(ICRF/J2000.0),dRA*cosD,d(DEC)/dt,SN.ang,SN.dist,...,delta,deldot,VmagSn,VmagOb,PlAng,ObsEcLon,ObsEcLat,GlxLon,GlxLat,Tru_Anom
0,2018-Dec-31 00:00:00.000,2458483.5,,,38.492878,14.808067,85.92155,-6.19821,94.13,0.0,...,1.452311,2.044452,25.28778,25.28778,-0.00911,40.825912,-0.278958,156.725443,-41.239274,64.6139
1,2019-Jan-01 00:00:00.000,2458484.5,,,39.051247,15.003341,85.77751,-6.24575,94.16,0.0,...,1.453495,2.054069,25.26810,25.26810,-0.00910,41.399454,-0.260651,157.204852,-40.793811,65.1878
2,2019-Jan-02 00:00:00.000,2458485.5,,,39.609720,15.196937,85.63319,-6.29235,94.20,0.0,...,1.454684,2.063466,25.24835,25.24835,-0.00909,41.972059,-0.242347,157.677090,-40.347123,65.7607
3,2019-Jan-03 00:00:00.000,2458486.5,,,40.168299,15.388839,85.48861,-6.33802,94.24,0.0,...,1.455878,2.072641,25.22853,25.22853,-0.00909,42.543725,-0.224048,158.142344,-39.899275,66.3326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88752,2261-Dec-29 00:00:00.000,2547235.5,,,110.592486,23.689155,69.77228,-6.30864,95.17,0.0,...,1.610426,1.698398,22.79285,22.79285,0.00644,108.796364,1.583575,194.558460,17.029624,131.5684
88753,2261-Dec-30 00:00:00.000,2547236.5,,,111.098025,23.630385,69.68975,-6.27737,95.15,0.0,...,1.611404,1.686066,22.77817,22.77817,0.00644,109.263244,1.590965,194.802054,17.434265,132.0358
88754,2261-Dec-31 00:00:00.000,2547237.5,,,111.602493,23.570052,69.60802,-6.24584,95.13,0.0,...,1.612374,1.673637,22.76361,22.76361,0.00644,109.729563,1.598240,195.046434,17.838121,132.5027
88755,2262-Jan-01 00:00:00.000,2547238.5,,,112.105888,23.508168,69.52708,-6.21406,95.11,0.0,...,1.613337,1.661112,22.74916,22.74916,0.00644,110.195327,1.605401,195.291629,18.241187,132.9690


### NOTE OUTLIER
271 September 29th, crazy declanation value

## Extract & rename relevant columns, export

In [63]:
# Extract and renam relevant columns, then export

eph = pd.DataFrame()
# eph['MJD'] = [Time(x).mjd for x in list(eph['date'].astype(str))]  # using astropy to convert
eph['MJD'] = df['Date_________JDUT'] - 2400000.5  # using definition   (https://en.wikipedia.org/wiki/Julian_day#Variants)
eph['date'] =  pd.to_datetime(df['Date__(UT)__HR:MN:SC.fff'])

eph['day'] = list(eph['MJD']) - eph['MJD'][0]
# eph.set_index('day', inplace=True)

eph['r'] =  df['r']
eph['theta'] = 90 - df['DEC_(ICRF/J2000.0)']
eph['phi'] = df['R.A._(ICRF/J2000.0)']

rs = list(eph['r'])
thetas = np.radians(list(eph['theta']))
phis = np.radians(list(eph['phi']))
eph['x'] = rs * sin(thetas) * cos(phis)
eph['y'] = rs * sin(thetas) * sin(phis)
eph['z'] = rs * cos(thetas)
# eph['r2'] = sqrt(eph['x']**2 + eph['y']**2 + eph['z']**2)  # sanity check

eph['day'] = eph['day'] - 1
eph.set_index('day', inplace=True)

eph.to_csv(OUTPUT)

pd.set_option('max_row', 9)
eph

,MJD,date,r,theta,phi,x,y,z
day,,,,,,,,
-1.0,58483.0,2018-12-31,1.452311,75.191933,38.492878,1.098950,0.873921,0.371184
0.0,58484.0,2019-01-01,1.453495,74.996659,39.051247,1.090280,0.884507,0.376274
1.0,58485.0,2019-01-02,1.454684,74.803063,39.609720,1.081506,0.895008,0.381327
2.0,58486.0,2019-01-03,1.455878,74.611161,40.168299,1.072627,0.905423,0.386344
...,...,...,...,...,...,...,...,...
88751.0,147235.0,2261-12-29,1.610426,66.310845,110.592486,-0.518690,1.380503,0.647028
88752.0,147236.0,2261-12-30,1.611404,66.369615,111.098025,-0.531411,1.377326,0.645907
88753.0,147237.0,2261-12-31,1.612374,66.429948,111.602493,-0.544095,1.374052,0.644740
88754.0,147238.0,2262-01-01,1.613337,66.491832,112.105888,-0.556740,1.370682,0.643527
